In [31]:
# Import dependencies
import pandas as pd
from sodapy import Socrata

# Load Data

In [32]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("chronicdata.cdc.gov", None)

In [33]:
# Return results as csv from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("swc5-untb", limit=2000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# View all columns in dataframes
pd.set_option('display.max_columns', None)  

# Display dataframe
results_df.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation,:@computed_region_bxsw_vy29,:@computed_region_he4y_prf8
0,2019,WY,Wyoming,Washakie,BRFSS,Health Outcomes,High cholesterol among adults aged >=18 years ...,%,Crude prevalence,32.1,31.2,33.1,7760,56043,HLTHOUT,HIGHCHOL,CrdPrv,High Cholesterol,"{'type': 'Point', 'coordinates': [-107.669052,...",14,3127
1,2020,WY,Wyoming,Laramie,BRFSS,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,23.8,21.5,26.2,100595,56021,RISKBEH,LPA,CrdPrv,Physical Inactivity,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119
2,2020,WY,Wyoming,Carbon,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,12.9,12.1,13.7,14711,56007,HLTHSTAT,MHLTH,CrdPrv,Mental Health,"{'type': 'Point', 'coordinates': [-106.9331526...",14,3081
3,2020,WY,Wyoming,Sublette,BRFSS,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,16.0,15.0,17.1,9856,56035,HLTHOUT,DEPRESSION,CrdPrv,Depression,"{'type': 'Point', 'coordinates': [-109.9161701...",14,3124
4,2020,WY,Wyoming,Uinta,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,3.0,2.7,3.4,20215,56041,HLTHOUT,STROKE,CrdPrv,Stroke,"{'type': 'Point', 'coordinates': [-110.5589468...",14,893


# General Data Overview

In [34]:
# View how many years worth of data we have
results_df['year'].unique()

array(['2019', '2020'], dtype=object)

In [35]:
# View null values by year
results_df.isnull().sum()

year                              0
stateabbr                         0
statedesc                         0
locationname                     60
datasource                        0
category                          0
measure                           0
data_value_unit                   0
data_value_type                   0
data_value                        0
low_confidence_limit              0
high_confidence_limit             0
totalpopulation                   0
locationid                        0
categoryid                        0
measureid                         0
datavaluetypeid                   0
short_question_text               0
geolocation                      60
:@computed_region_bxsw_vy29    1192
:@computed_region_he4y_prf8     660
dtype: int64

In [36]:
# Check data types for results_df
results_df.dtypes

year                           object
stateabbr                      object
statedesc                      object
locationname                   object
datasource                     object
category                       object
measure                        object
data_value_unit                object
data_value_type                object
data_value                     object
low_confidence_limit           object
high_confidence_limit          object
totalpopulation                object
locationid                     object
categoryid                     object
measureid                      object
datavaluetypeid                object
short_question_text            object
geolocation                    object
:@computed_region_bxsw_vy29    object
:@computed_region_he4y_prf8    object
dtype: object

In [37]:
# View health categories - (which include metrics we could use as features for machine learning)
results_df['category'].unique()

array(['Health Outcomes', 'Health Risk Behaviors', 'Health Status',
       'Prevention'], dtype=object)

In [38]:
# View contributing health metrics (possible features for machine learning)
results_df.loc[results_df['category'].isin(['Prevention', 'Health Risk Behaviors', 'Health Status'])]['measureid'].unique()

array(['LPA', 'MHLTH', 'DENTAL', 'BINGE', 'COREW', 'CERVICAL', 'PHLTH',
       'BPMED', 'COLON_SCREEN', 'CHOLSCREEN', 'COREM', 'SLEEP',
       'CSMOKING', 'MAMMOUSE', 'ACCESS2', 'GHLTH', 'CHECKUP'],
      dtype=object)

In [39]:
# View disease outcomes (possible targets for machine learning)
# This analysis will focus on COPD (Chronic Obstructive Pulmonary Disease)
# However, this notebook will make it easy to analyze other diseases
results_df['measureid'].unique()

array(['HIGHCHOL', 'LPA', 'MHLTH', 'DEPRESSION', 'STROKE', 'CASTHMA',
       'BPHIGH', 'DIABETES', 'DENTAL', 'BINGE', 'COPD', 'COREW',
       'CERVICAL', 'CHD', 'PHLTH', 'BPMED', 'COLON_SCREEN', 'KIDNEY',
       'CHOLSCREEN', 'COREM', 'SLEEP', 'CSMOKING', 'MAMMOUSE',
       'ARTHRITIS', 'ACCESS2', 'TEETHLOST', 'CANCER', 'GHLTH', 'CHECKUP',
       'OBESITY'], dtype=object)

# Prepare Dataframe

In [40]:
# Create a list of columns from results_df we want to convert from object to numeric
# This will allow us to perform calculations on the data
numeric_list = ['year', 'data_value', 'low_confidence_limit', 'high_confidence_limit', 'totalpopulation']

# Convert columns in numeric_list to numeric data type in results_df
results_df[numeric_list] = results_df[numeric_list].apply(pd.to_numeric, errors='coerce')

# View data types for results_df
results_df.dtypes

year                             int64
stateabbr                       object
statedesc                       object
locationname                    object
datasource                      object
category                        object
measure                         object
data_value_unit                 object
data_value_type                 object
data_value                     float64
low_confidence_limit           float64
high_confidence_limit          float64
totalpopulation                  int64
locationid                      object
categoryid                      object
measureid                       object
datavaluetypeid                 object
short_question_text             object
geolocation                     object
:@computed_region_bxsw_vy29     object
:@computed_region_he4y_prf8     object
dtype: object

In [41]:
# Create a list of the health metrics which will become our features and make them into dataframes
# And print the feature and the measure of each dataframe as a key to more easily understand the data

# Add the health outcomes to a list
features_list = results_df.loc[results_df['category'].isin(['Prevention', 'Health Risk Behaviors', 'Health Status'])]['measureid'].unique()

# Create unique dataframes for each feature in the features_list
for feature in features_list:
    globals()[feature + "_df"] = results_df.loc[results_df['measureid'] == feature]

# Create a list that contains the names of the dataframes made in the step above
dataframes_list = [feature + "_df" for feature in features_list]

# Print the feature and the measure of each dataframe
for dataframe in dataframes_list:
    print(f"{dataframe} - {globals()[dataframe]['measure'].unique()}")

LPA_df - ['No leisure-time physical activity among adults aged >=18 years']
MHLTH_df - ['Mental health not good for >=14 days among adults aged >=18 years']
DENTAL_df - ['Visits to dentist or dental clinic among adults aged >=18 years']
BINGE_df - ['Binge drinking among adults aged >=18 years']
COREW_df - ['Older adult women aged >=65 years who are up to date on a core set of clinical preventive services: Flu shot past year, PPV shot ever, Colorectal cancer screening, and Mammogram past 2 years']
CERVICAL_df - ['Cervical cancer screening among adult women aged 21-65 years']
PHLTH_df - ['Physical health not good for >=14 days among adults aged >=18 years']
BPMED_df - ['Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure']
COLON_SCREEN_df - ['Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years']
CHOLSCREEN_df - ['Cholesterol screening among adults aged >=18 years']
COREM_df - ['Older adult men aged >=65 ye

In [42]:
# Create the cancer dataframe by filtering results_df which will be our taget
cancer_df = results_df.loc[results_df["measureid"] == "CANCER", :]

# Display dataframe and view all columns
cancer_df.head(3)

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation,:@computed_region_bxsw_vy29,:@computed_region_he4y_prf8
77,2020,WY,Wyoming,Niobrara,BRFSS,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,%,Crude prevalence,8.4,8.0,8.9,2275,56027,HLTHOUT,CANCER,CrdPrv,Cancer (except skin),"{'type': 'Point', 'coordinates': [-104.4683727...",14,3121
89,2020,WY,Wyoming,Sheridan,BRFSS,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,%,Age-adjusted prevalence,6.0,5.7,6.3,30863,56033,HLTHOUT,CANCER,AgeAdjPrv,Cancer (except skin),"{'type': 'Point', 'coordinates': [-106.8812114...",14,3123
133,2020,WY,Wyoming,Lincoln,BRFSS,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,%,Crude prevalence,7.7,7.3,8.1,20253,56023,HLTHOUT,CANCER,CrdPrv,Cancer (except skin),"{'type': 'Point', 'coordinates': [-110.6829614...",14,3120


In [43]:
# View dataframes list
dataframes_list

['LPA_df',
 'MHLTH_df',
 'DENTAL_df',
 'BINGE_df',
 'COREW_df',
 'CERVICAL_df',
 'PHLTH_df',
 'BPMED_df',
 'COLON_SCREEN_df',
 'CHOLSCREEN_df',
 'COREM_df',
 'SLEEP_df',
 'CSMOKING_df',
 'MAMMOUSE_df',
 'ACCESS2_df',
 'GHLTH_df',
 'CHECKUP_df']

In [44]:
# Add copd_df to dataframes_list
dataframes_list.append("cancer_df")

# Filter dataframes to only include datavaluetypeid AgeAdjPrv
for dataframe in dataframes_list:
    globals()[dataframe] = globals()[dataframe].loc[globals()[dataframe]['datavaluetypeid'] == 'AgeAdjPrv']

# Change the name of data_value to the name of the measureid + data_value for all dataframes
for dataframe in dataframes_list:
    globals()[dataframe] = globals()[dataframe].rename(columns={'data_value': globals()[dataframe]['measureid'].unique()[0] + '_data_value'})


# Drop unnecessary columns for all dataframes in dataframes_list
for dataframe in dataframes_list:
    globals()[dataframe] = globals()[dataframe].drop(columns=['statedesc', 'datasource', 'category', 'measure', 
                                                              'data_value_unit', 'locationid', 'categoryid', 
                                                              'measureid', 'data_value_type', 'low_confidence_limit', 
                                                              'high_confidence_limit', 'short_question_text', 'geolocation', 
                                                              'datavaluetypeid', ':@computed_region_bxsw_vy29', 
                                                              ':@computed_region_he4y_prf8'], axis=1)

# Display dataframe and view all columns
cancer_df.head(6)

,year,stateabbr,locationname,CANCER_data_value,totalpopulation
89,2020,WY,Sheridan,6.0,30863
365,2020,WY,Goshen,6.0,13235
500,2020,WY,Uinta,5.9,20215
604,2020,WY,Park,6.0,29331
624,2020,WY,Big Horn,6.0,11575
672,2020,WY,Carbon,5.8,14711


In [47]:
# View dataframes list
dataframes_list

['LPA_df',
 'MHLTH_df',
 'DENTAL_df',
 'BINGE_df',
 'COREW_df',
 'CERVICAL_df',
 'PHLTH_df',
 'BPMED_df',
 'COLON_SCREEN_df',
 'CHOLSCREEN_df',
 'COREM_df',
 'SLEEP_df',
 'CSMOKING_df',
 'MAMMOUSE_df',
 'ACCESS2_df',
 'GHLTH_df',
 'CHECKUP_df',
 'cancer_df']

In [50]:
# Copy the copd_df dataframe to a new dataframe
ml_df = cancer_df.copy()

# Merge the dataframes in dataframes_list to ml_df
for dataframe in dataframes_list:
    ml_df = ml_df.merge(globals()[dataframe], how='left', on=['year', 'totalpopulation', 'stateabbr', 'locationname'])

# Rename the column COPD_data_value_x to COPD_data_value
ml_df = ml_df.rename(columns={'CANCER_data_value_x': 'CANCER_data_value'})

# Drop the column COPD_data_value_y
ml_df = ml_df.drop(columns=['CANCER_data_value_y'], axis=1)

# Display dataframe and view all columns
ml_df.head(3)




,year,stateabbr,locationname,CANCER_data_value,totalpopulation,LPA_data_value,MHLTH_data_value,DENTAL_data_value,BINGE_data_value,COREW_data_value,CERVICAL_data_value,PHLTH_data_value,BPMED_data_value,COLON_SCREEN_data_value,CHOLSCREEN_data_value,COREM_data_value,SLEEP_data_value,CSMOKING_data_value,MAMMOUSE_data_value,ACCESS2_data_value,GHLTH_data_value,CHECKUP_data_value
0,2020,WY,Sheridan,6.0,30863,20.4,13.3,65.3,19.3,32.3,81.9,8.4,NaN,63.4,NaN,50.0,31.9,17.6,69.2,14.7,10.8,64.0
1,2020,WY,Goshen,6.0,13235,24.3,13.3,63.3,18.8,30.5,81.2,9.0,NaN,56.9,NaN,34.3,32.3,18.4,64.3,17.7,12.2,63.6
2,2020,WY,Uinta,5.9,20215,23.4,13.9,63.7,17.3,28.7,80.8,9.2,NaN,62.2,NaN,34.3,35.3,18.9,61.7,17.2,11.9,63.2


In [51]:
# Get summary statistics for all columns
ml_df.describe()

,year,CANCER_data_value,totalpopulation,LPA_data_value,MHLTH_data_value,DENTAL_data_value,BINGE_data_value,COREW_data_value,CERVICAL_data_value,PHLTH_data_value,BPMED_data_value,COLON_SCREEN_data_value,CHOLSCREEN_data_value,COREM_data_value,SLEEP_data_value,CSMOKING_data_value,MAMMOUSE_data_value,ACCESS2_data_value,GHLTH_data_value,CHECKUP_data_value
count,3144.0,3144.000000,3.144000e+03,3144.000000,3144.000000,3144.000000,3144.000000,3144.000000,3144.000000,3144.000000,0.0,3144.000000,0.0,3144.000000,3144.000000,3144.000000,3144.000000,3144.000000,3144.000000,3144.000000
mean,2020.0,5.902926,2.102205e+05,25.709796,15.730184,58.988963,17.834669,36.866349,81.565331,10.963836,NaN,67.734765,NaN,42.759351,34.471947,20.034256,70.444593,16.797328,16.032029,72.762309
std,0.0,0.285336,5.918720e+06,5.192709,2.040912,7.488897,3.020080,4.477454,2.344340,2.152859,NaN,4.738214,NaN,5.143715,3.637180,4.105682,3.959298,6.606075,4.442595,4.045601
min,2020.0,4.300000,8.700000e+01,10.200000,8.300000,33.200000,8.200000,16.200000,68.400000,5.600000,NaN,49.000000,NaN,19.800000,23.800000,5.800000,50.600000,5.700000,6.500000,58.300000
25%,2020.0,5.800000,1.088425e+04,22.000000,14.300000,53.600000,15.700000,33.900000,80.100000,9.300000,NaN,64.800000,NaN,39.300000,31.900000,17.500000,67.900000,12.100000,12.600000,69.800000
50%,2020.0,6.000000,2.562050e+04,25.200000,15.700000,59.800000,17.600000,36.800000,81.800000,10.800000,NaN,68.100000,NaN,42.700000,34.400000,19.850000,70.700000,15.000000,15.300000,73.200000
75%,2020.0,6.100000,6.829725e+04,29.200000,17.200000,64.525000,19.700000,39.800000,83.200000,12.400000,NaN,71.300000,NaN,46.000000,36.800000,22.600000,73.300000,19.500000,19.000000,75.600000
max,2020.0,6.400000,3.314493e+08,47.200000,23.300000,80.700000,27.600000,53.400000,87.700000,20.900000,NaN,82.100000,NaN,60.600000,48.400000,41.100000,83.400000,53.300000,36.800000,82.600000


# Machine Learning

In [52]:
# Import dependencies for machine learning
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'tensorflow'

In [53]:
# View ml_df
ml_df.head(3)

,year,stateabbr,locationname,CANCER_data_value,totalpopulation,LPA_data_value,MHLTH_data_value,DENTAL_data_value,BINGE_data_value,COREW_data_value,CERVICAL_data_value,PHLTH_data_value,BPMED_data_value,COLON_SCREEN_data_value,CHOLSCREEN_data_value,COREM_data_value,SLEEP_data_value,CSMOKING_data_value,MAMMOUSE_data_value,ACCESS2_data_value,GHLTH_data_value,CHECKUP_data_value
0,2020,WY,Sheridan,6.0,30863,20.4,13.3,65.3,19.3,32.3,81.9,8.4,NaN,63.4,NaN,50.0,31.9,17.6,69.2,14.7,10.8,64.0
1,2020,WY,Goshen,6.0,13235,24.3,13.3,63.3,18.8,30.5,81.2,9.0,NaN,56.9,NaN,34.3,32.3,18.4,64.3,17.7,12.2,63.6
2,2020,WY,Uinta,5.9,20215,23.4,13.9,63.7,17.3,28.7,80.8,9.2,NaN,62.2,NaN,34.3,35.3,18.9,61.7,17.2,11.9,63.2


In [54]:
# Check for null values
ml_df.isnull().sum()

year                          0
stateabbr                     0
locationname                  1
CANCER_data_value             0
totalpopulation               0
LPA_data_value                0
MHLTH_data_value              0
DENTAL_data_value             0
BINGE_data_value              0
COREW_data_value              0
CERVICAL_data_value           0
PHLTH_data_value              0
BPMED_data_value           3144
COLON_SCREEN_data_value       0
CHOLSCREEN_data_value      3144
COREM_data_value              0
SLEEP_data_value              0
CSMOKING_data_value           0
MAMMOUSE_data_value           0
ACCESS2_data_value            0
GHLTH_data_value              0
CHECKUP_data_value            0
dtype: int64

In [55]:
# Drop BPMED_data_value and CHOLSCREEN_data_value columns
ml_df = ml_df.drop(columns=['BPMED_data_value', 'CHOLSCREEN_data_value'], axis=1)

In [56]:
# View the null value in the locationname column
ml_df.loc[ml_df['locationname'].isnull()]

,year,stateabbr,locationname,CANCER_data_value,totalpopulation,LPA_data_value,MHLTH_data_value,DENTAL_data_value,BINGE_data_value,COREW_data_value,CERVICAL_data_value,PHLTH_data_value,COLON_SCREEN_data_value,COREM_data_value,SLEEP_data_value,CSMOKING_data_value,MAMMOUSE_data_value,ACCESS2_data_value,GHLTH_data_value,CHECKUP_data_value
365,2020,US,NaN,5.5,331449281,22.9,13.9,64.5,16.7,37.4,83.7,9.4,70.6,44.0,33.3,14.6,77.8,13.5,13.7,73.0


In [57]:
# Drop the null value in the locationname column
ml_df = ml_df.dropna()

# Check for null values
ml_df.isnull().sum()

year                       0
stateabbr                  0
locationname               0
CANCER_data_value          0
totalpopulation            0
LPA_data_value             0
MHLTH_data_value           0
DENTAL_data_value          0
BINGE_data_value           0
COREW_data_value           0
CERVICAL_data_value        0
PHLTH_data_value           0
COLON_SCREEN_data_value    0
COREM_data_value           0
SLEEP_data_value           0
CSMOKING_data_value        0
MAMMOUSE_data_value        0
ACCESS2_data_value         0
GHLTH_data_value           0
CHECKUP_data_value         0
dtype: int64

In [58]:
# Create label encoder instance
le = LabelEncoder()

# Create a copy of ml_df
ml_df_encoded = ml_df.copy()

# Generate our categorical variable lists
dataset_cat = ml_df_encoded.dtypes[ml_df_encoded.dtypes == "object"].index.tolist()

# View the list of categorical variables
dataset_cat

NameError: name 'LabelEncoder' is not defined

In [59]:
# For loop to encode text columns to numerical values
for textColumn in dataset_cat:
    ml_df_encoded[textColumn] = le.fit_transform(ml_df_encoded[textColumn])

# View ml_df
ml_df_encoded.head(3)

NameError: name 'dataset_cat' is not defined

In [60]:
# Create list of columns to be our features
feature_cols = [i for i in ml_df_encoded.columns if i not in ('CANCER_data_value')]
X = ml_df_encoded[feature_cols]

# Create our target column
y = ml_df_encoded['CANCER_data_value']

NameError: name 'ml_df_encoded' is not defined